# Implementing Transformer Models
## Practical III
Carel van Niekerk & Hsien-Chin Lin

27-31.10.2025

---

In previous practicals, we delved into the attention mechanism, which serves as the foundation of transformer-style models. We noted that such mechanisms necessitate the representation of text as numerical vectors. In this session, we will investigate word tokenizers, which are methods for converting words into meaningful subword units termed as 'tokens'. Specifically, we will implement a basic Byte Pair Encoding (BPE) tokenizer to gain insights into the workings of this kind of tokenizer.

### 1. Tokenizers

Word tokenizers are used to split text into tokens, which can be words or subwords. In this practical we investigate the BPE tokenizer. BPE is a simple algorithm that iteratively replaces the most frequent pair of characters in a text with a new character. This process is repeated until a predefined number of tokens is reached. The BPE algorithm is described in the following paper: [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/pdf/1508.07909.pdf).

### 2. The Byte-Pair Encoding (BPE) Tokenizer

The BPE algorithm is implemented in the following steps:

#### 2.1. Building the base vocabulary

The base vocabulary is a set of all the characters present in the data. To obtain the base vocabulary, we first find the set of all unique words in a corpus. We then find the set of all unique characters in theses words.

For example, given the following set of words:

`['hug', 'pug', 'pun', 'bun', 'hugs']`

The base vocabulary is:

`['h', 'u', 'g', 'p', 'n', 'b', 's']`

#### 2.2. Building the BPE vocabulary

Once we have the base vocabulary, we learn a set of merges, these are rules indicating which characters should be merged. Each merge becomes a new token in the vocabulary. The merges are learned by iteratively finding the most frequent pair of characters in the data and merging them. This process is repeated until a predefined vocabulary size is reached.

Let us assume that each of the above words has a frequency of:

`{'hug': 10, 'pug': 5, 'pun': 12, 'bun': 4, 'hugs': 5}`

We can now compute the co-occurrence frequencies of all tokens in the vocabulary:

`{('h', 'u'): 15, ('u', 'g'): 20, ('p', 'u'): 17, ('u', 'n'): 16, ('b', 'u'): 4, ('g', 's'): 5}`

We see that the characters `('u', 'g')` co-occur the most. We create the merge rule `('u', 'g')` resulting in the new token 'ug'. We can now update the vocabulary and co-occurrence frequencies to:

`['h', 'u', 'g', 'p', 'n', 'b', 's', 'ug']`

`{('p', 'u'): 12, ('u', 'n'): 16, ('b', 'u'): 4, ('h', 'ug'): 15, ('p', 'ug'): 5, ('ug', 's'): 5}`

The next merge rule is `('u', 'n')` resulting in the token 'un'.

If we stop here we obtain the vocabulary:

`['h', 'u', 'g', 'p', 'n', 'b', 's', 'ug', 'un']`

and the set of merge rules:

`{('u', 'g'): 'ug', ('u', 'n'): 'un'}`.

#### 2.3. Encoding a word

Based on this vocabulary we can now encode a word. First the word, for example 'pugs', is split into characters:

`['p', 'u', 'g', 's']`

Then the merge rules are applied to the word (here 'u' and 'g' are combined to become 'ug'):
`['p', 'ug', 's']`

Finally, the word is encoded as a sequence of tokens:
`['p', 'ug', 's']`.

# Exercises

1. Implement the BPE tokenizer module. The module should be able to extract the vocubulary from a corpus of text.
2. Given the corpus below train your BPE tokenizer. Use a vocabulary size of 64.

```python
[
    "Machine learning helps in understanding complex patterns.",
    "Learning machine languages can be complex yet rewarding.",
    "Natural language processing unlocks valuable insights from data.",
    "Processing language naturally is a valuable skill in machine learning.",
    "Understanding natural language is crucial in machine learning."
]
```

3. Using the BPETokenizer implementation of Huggingface ([more info](https://pypi.org/project/tokenizers/)) train a BPE tokenizer using the above corpus. Use a vocabulary of size 295 (due to larger default base vocab of this implmentation).
4. Tokenize the following sentence: "Machine learning is a subset of artificial intelligence." using both your implementation and the Huggingface implementation

### Additional Material
- [Huggingface tutorial on BPE](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt)

In [2]:
import sys
sys.path.append('d:/Code/implementingtransformers')

from transformer_project.tokenizer import BPETokenizer

corpus = [
    "Machine learning helps in understanding complex patterns.",
    "Learning machine languages can be complex yet rewarding.",
    "Natural language processing unlocks valuable insights from data.",
    "Processing language naturally is a valuable skill in machine learning.",
    "Understanding natural language is crucial in machine learning."
]

tokenizer = BPETokenizer(vocab_size=64)
vocab, merges = tokenizer.train(corpus)

print("Vocabulary size:", len(vocab))
print("\nVocabulary:", vocab)
print("\nMerge rules:")
for pair, merged in merges.items():
    print(f"  {pair} -> {merged}")

Vocabulary size: 64

Vocabulary: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'k': 9, 'l': 10, 'm': 11, 'n': 12, 'o': 13, 'p': 14, 'r': 15, 's': 16, 't': 17, 'u': 18, 'v': 19, 'w': 20, 'x': 21, 'y': 22, 'in': 23, 'ing': 24, 'le': 25, 'an': 26, 'ua': 27, 'al': 28, 'ar': 29, 'at': 30, 'ma': 31, 'mac': 32, 'mach': 33, 'machin': 34, 'machine': 35, 'lear': 36, 'learn': 37, 'learning': 38, 'lan': 39, 'lang': 40, 'langua': 41, 'languag': 42, 'language': 43, 'un': 44, 'er': 45, 'ding': 46, 'om': 47, 'nat': 48, 'natu': 49, 'natur': 50, 'natural': 51, 'oc': 52, 'und': 53, 'under': 54, 'unders': 55, 'underst': 56, 'understan': 57, 'understanding': 58, 'com': 59, 'comp': 60, 'comple': 61, 'complex': 62, 'pr': 63}

Merge rules:
  ('i', 'n') -> in
  ('in', 'g') -> ing
  ('l', 'e') -> le
  ('a', 'n') -> an
  ('u', 'a') -> ua
  ('a', 'l') -> al
  ('a', 'r') -> ar
  ('a', 't') -> at
  ('m', 'a') -> ma
  ('ma', 'c') -> mac
  ('mac', 'h') -> mach
  ('mach', 'in') -> machin
  (

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

hf_tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
hf_tokenizer.pre_tokenizer = Whitespace()

trainer = BpeTrainer(vocab_size=295, special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"])
hf_tokenizer.train_from_iterator(corpus, trainer)

print("HuggingFace BPE Vocabulary size:", hf_tokenizer.get_vocab_size())

HuggingFace BPE Vocabulary size: 135


In [4]:
test_sentence = "Machine learning is a subset of artificial intelligence."

print("Custom BPE tokenizer:")
custom_tokens = tokenizer.tokenize(test_sentence)
print(f"  Tokens: {custom_tokens}")
print(f"  Encoded: {tokenizer.encode(test_sentence)}")

print("\nHuggingFace BPE tokenizer:")
hf_output = hf_tokenizer.encode(test_sentence)
print(f"  Tokens: {hf_output.tokens}")
print(f"  Encoded: {hf_output.ids}")

Custom BPE tokenizer:
  Tokens: ['machine', 'learning', 'i', 's', 'a', 's', 'u', 'b', 's', 'e', 't', 'o', 'f', 'ar', 't', 'i', 'f', 'i', 'c', 'i', 'al', 'in', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 'c', 'e']
  Encoded: [35, 38, 8, 16, 0, 16, 18, 1, 16, 4, 17, 13, 5, 29, 17, 8, 5, 8, 2, 8, 28, 23, 17, 4, 10, 10, 8, 6, 4, 12, 2, 4]

HuggingFace BPE tokenizer:
  Tokens: ['Machine', 'learning', 'is', 'a', 's', 'u', 'b', 's', 'et', 'o', 'f', 'ar', 't', 'i', 'f', 'i', 'ci', 'al', 'in', 't', 'el', 'l', 'i', 'ge', 'n', 'c', 'e', '.']
  Encoded: [84, 60, 66, 11, 27, 29, 12, 27, 94, 24, 16, 40, 28, 19, 16, 19, 89, 38, 34, 28, 93, 21, 19, 43, 23, 13, 15, 5]
